In [29]:
import argparse
import pysam
from pathlib import Path
import pandas as pd
import numpy as np

In [30]:
from IPython.display import clear_output

# Clear the output of the current notebook
clear_output(wait=True)

# If you want to clear outputs from all cells, run this:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.instance().run_line_magic('reset', '-f')

In [31]:
strandedness = []

import argparse
import pysam
from pathlib import Path
import pandas as pd
import numpy as np

import pysam
fasta = pysam.FastaFile("hg38.fa")
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

from IPython.utils import io

In [32]:
import pysam
from pathlib import Path

#contains the locations of all the guides (lifted)
altc = Path('small_bams_again') / 'beds'  / 'altc_lifted.bed'
alt7 = Path('small_bams_again') / 'beds'  / 'alt7_lifted.bed'
rex4 = Path('small_bams_again') / 'beds'  / 'rex4_lifted.bed'
rexc = Path('small_bams_again') / 'beds' / 'rexc_lifted.bed'

#all the dfs (at the moment) 

# ALT7 bams (not yet simulating) 
ALT7_1_BAMs = Path('small_bams_again') / 'ALT7_realigned' / 'rep1' / '7_prime_rep1_merged_sorted.bam'
ALT7_2_BAMs = Path('small_bams_again') / 'ALT7_realigned' / 'rep2' / '7_prime_rep2_merged_sorted.bam'

# ALTC bams
ALTC_1_BAMs = Path('small_bams_again') / 'ALTC_realigned' / 'rep1' / 'c_prime_rep1_merged_sorted.bam'
ALTC_2_BAMs = Path('small_bams_again') / 'ALTC_realigned' / 'rep2' / 'c_prime_rep2_merged_sorted.bam'

# REXC bams
REXC_1_BAMs = Path('small_bams_again') / 'REXC_realigned' / 'rep1' / 'c_rep1_merged_sorted.bam'
REXC_2_BAMs = Path('small_bams_again') / 'REXC_realigned' / 'rep2' / 'c_rep2_merged_sorted.bam'

# REX4 bams
REX4_2_BAMs = Path('small_bams_again') / 'REX4_realigned' / 'rep1' / '4_rep1_merged_sorted.bam'
REX4_3_BAMs = Path('small_bams_again') / 'REX4_realigned' / 'rep2' / '4_rep2_merged_sorted.bam'


ALTC_NO_NUNC = Path('small_bams_again') / 'ALTC_realigned' / 'nuc' / 'c_nuc_merged_sorted.bam'
REX4_NO_NUNC = Path('small_bams_again') / 'REX4_realigned' / 'nuc' / '4_nuc_merged_sorted.bam'
ALT7_NO_NUNC = Path('small_bams_again') / 'ALT7_realigned' / 'nuc' / '7_nuc_merged_sorted.bam'
REXC_NO_NUNC = Path('small_bams_again') / 'REXC_realigned' / 'nuc' / 'c_nuc_merged_sorted.bam'



# Function to read BAM files from a directory and store in a dictionary
#currently the BAM file is stored in memory (which could lead to memory issues) -->
#rather than save the entire BAM file (and open it - save the path to the PAM file and do downstream analysis)
def read_bam_files_to_dict(directory):
    bam_files_dict = {}
    
    #sort the directory by name
    sorted_bam_files = sorted(directory.glob('*.bam'), key=lambda f: f.name)
    
    for bam_file in sorted_bam_files:  # Filter for .bam files
        #bam = pysam.AlignmentFile(bam_file, "rb") 
        bam_files_dict[bam_file.name] = bam_file.resolve() # Store filename as key, BAM object as value
    return bam_files_dict

# Read BAM files from each directory

#ALTC
bam_files_AltC_1 = read_bam_files_to_dict(ALTC_1_BAMs)
bam_files_AltC_2 = read_bam_files_to_dict(ALTC_2_BAMs)

#ALT7
bam_files_Alt7_1 = read_bam_files_to_dict(ALT7_1_BAMs)
bam_files_Alt7_2 = read_bam_files_to_dict(ALT7_2_BAMs)

#REX4
bam_files_REX4_2 = read_bam_files_to_dict(REX4_2_BAMs)
bam_files_REX4_3 = read_bam_files_to_dict(REX4_3_BAMs)

#REXC
bam_files_REXC_1 = read_bam_files_to_dict(REXC_1_BAMs)
bam_files_REXC_2 = read_bam_files_to_dict(REXC_2_BAMs)

#NO_NUNC 
bam_files_NO_NUNC_ALTC = read_bam_files_to_dict(ALTC_NO_NUNC)
bam_files_NO_NUNC_ALT7 = read_bam_files_to_dict(ALT7_NO_NUNC)
bam_files_NO_NUNC_REX4 = read_bam_files_to_dict(REX4_NO_NUNC)
bam_files_NO_NUNC_REXC = read_bam_files_to_dict(REXC_NO_NUNC)


#THE BAM FILES TO CONSIDER
#refactor to account for a merged bam instead
all_bams = {'ALTC': [ALTC_1_BAMs, ALTC_2_BAMs],
            'ALT7': [ALT7_1_BAMs, ALT7_2_BAMs],
            'REX4': [REX4_2_BAMs, REX4_3_BAMs],
            'REXC': [REXC_1_BAMs, REXC_2_BAMs]}

#THE CONTROL BAMS
all_controls = {'ALTC':ALTC_NO_NUNC,
                'ALT7':ALT7_NO_NUNC,
                'REX4':REX4_NO_NUNC,
                'REXC':REXC_NO_NUNC}

#TSVs for the locations of the potential locations
all_tsvs = {'ALTC':pd.read_csv(altc, sep='\t',  names=['chrom','start','end','name','na','na2','REGION'])[:],
                'ALT7':pd.read_csv(alt7, sep='\t',  names=['chrom','start','end','na','na2','REGION'])[:],
                'REX4':pd.read_csv(rex4, sep='\t',  names=['chrom','start','end','na','na2','REGION'])[:],
                'REXC':pd.read_csv(rexc, sep='\t',  names=['chrom','start','end','na','na2','REGION'])[:]}




#NO_NUNC 
bam_files_NO_NUNC_ALTC = read_bam_files_to_dict(ALTC_NO_NUNC)
bam_files_NO_NUNC_ALT7 = read_bam_files_to_dict(ALT7_NO_NUNC)
bam_files_NO_NUNC_REX4 = read_bam_files_to_dict(REX4_NO_NUNC)
bam_files_NO_NUNC_REXC = read_bam_files_to_dict(REXC_NO_NUNC)

all_controls = {'ALTC':ALTC_NO_NUNC,
                'ALT7':ALT7_NO_NUNC,
                'REX4':REX4_NO_NUNC,
                'REXC':REXC_NO_NUNC}

#iterate through all the tsvs and make a "REGION' entry
for condition in all_tsvs:
    all_tsvs[condition]['REGION'] = ['REGION_' + str(i) + '.bam' for i in np.arange(0,  all_tsvs[condition]['REGION'].shape[0])]


In [33]:
the_read = None

#this finds the deletions given a specific inbam (don't need to modify)
def find_deletion_locs(inbam, chrom, start_loc, end_loc,strand):
    deletions = []
    insertions = []
    #table of all indels (-) for a feletion (+) for an insertion
    indels = []
    count = 0
    
    #iterate through the inbam 
    for read in inbam.fetch(chrom, start_loc, end_loc):
        
        deletion_lst = []
        insertion_lst = []
        indel_lst = []

        count += 1
        ref_start = read.reference_start  # Correct attribute for reference start position

        #if the start of the read is soft clipped
        if read.is_unmapped:
            print(chrom, start_loc, end_loc)
            print('UNMAPPED')
            break
        if read.cigartuples[0][0] == 4:
            #make the location of the read start at the reference minus the length of the soft clipping as reference_start doesn't
            #otherwise include the soft clipped bases 
            ref_start = ref_start - read.cigartuples[0][1] 
        
        cigar = read.cigartuples
    
        #the bases traversed - where we are in the read 
        l_read = 0
        
        #the exact start position of the read - where we are in the reference 
        l = ref_start
        segment_index = 0

        # if start_loc == 27573507:
        #     print(cigar)
        #     print(ref_start)

        #go through now the entire cigar array
        for op, length in cigar:

            if strand == '-':
                #on the 3' end
                if length + l > start_loc and l < start_loc:
                    if op == 2:
                        # print("Deletion found")
                        deletion_lst.append(["deletions extending passed region", length, (l, l+length)])
                        indel_lst.append(["indels", -1 * length])
                #on the 5' end
                elif l + length > end_loc and l < end_loc:
                    if op == 2:
                        # print("deletion found")
                        deletion_lst.append(["deletions extending passed region", length, (l, l+length)])
                        indel_lst.append(["indels", -1 * length])
                #starts in the region
                elif l >= start_loc and l <= end_loc:
                    if op == 2:
                        # print("deletion found")
                        deletion_lst.append(["deletions", length, (l, l+length)])
                        indel_lst.append(["indels", -1 * length])
                #deletuon ends in the region
                elif l + length >= start_loc and l + length  <= end_loc:
                    if op == 2:
                        deletion_lst.append(["deletions", length, (l, l+length)])
                        indel_lst.append(["indels", -1 * length])

            #deletion should be on the 3' end (at the end)
            if strand == '+' or strand == 'unknown':
                # if start_loc == 27573507:
                #     print('got at least the strand')
                if length + l > end_loc and l < end_loc:
                    if op == 2:
                        # print("deletion found")
                        deletion_lst.append(["deletions extending passed region", length, (l, l+length)])
                        indel_lst.append(["indels", -1 * length])
                        #may 8th - put an equal sign 
                elif length + l > start_loc and l < start_loc:
                    if op == 2:
                        # print("Deletion found")
                        deletion_lst.append(["deletions extending passed region", length, (l, l+length)])
                        indel_lst.append(["indels", -1 * length])
                #deletion starts in the region
                elif l >= start_loc and l  <= end_loc:
                    if op == 2:
                        # print("deletion found")
                        deletion_lst.append(["deletions", length, (l, l+length)])
                        indel_lst.append(["indels", -1 * length])
                #deletion ends in the region
                elif l + length >= start_loc and l + length  <= end_loc:
                    if op == 2:
                        deletion_lst.append(["deletions", length, (l, l+length)])
                        indel_lst.append(["indels", -1 * length])

            #insertion ends in the guide loc
            if l + length < end_loc and l + length > start_loc :
                if op == 1:
                    insertion_lst.append(["insertions", length, (l, l+length)])
                    indel_lst.append(["indels", length])
                    
            #insertion begins in the guide loc
            if l  < end_loc and l > start_loc :
                if op == 1:
                    insertion_lst.append(["insertions", length, (l, l+length)])
                    indel_lst.append(["indels", length])
                
            
            '''below traverses the region of the read we are considering'''
            if op == 0:  # NOT SURE WHAT THE MEANING OF THIS IS
                l_read += length
                l += length

            #THIS SEEMS STRANGE -- BUT THIS ALSO WORKED? SO NOW I AM CONFUSED ARGH
            elif op == 1:  # Insertion (this is correct)
                l_read += length
                
            elif op == 2:  # Deletion (this is correct)
                # print('here')
                # if start_loc == 27573507:
                #     print(l, l+length)
                #     print(start_loc, end_loc)
                l += length 
                
            elif op == 3:  # Skipped region
                l += length #moving ahead in the reference
                
            elif op == 4:  # Soft clipping
                l_read += length
                l += length #
                
            elif op == 5:  # Hard clipping NOT SURE 
                pass
            elif op == 6:  # Padding NOT SURE 
                pass
            elif op == 7:  # Segment equal (this is correct)
                l_read += length
                l += length
                
            elif op == 8:  # seems to be sequence mismatcj - they both advance then NOT SURE
                l_read += length
                l += length
                
            else:
                raise ValueError(f"Unknown CIGAR operation code {op}")

            #if - once adding the oppcode have suprased the end of the read - break
            if l > end_loc:
                if len(deletion_lst) > 0:
                    deletions.append(deletion_lst)
                if len(insertion_lst) > 0:
                    insertions.append(insertion_lst)
                if len(indel_lst) > 0:
                    indels.append(indel_lst)
                break
                
            segment_index += 1
            

    #returns a list of all the deletions and all the insertions and a total indel lst
    return deletions, insertions, indels
            
    # Close the BAM file
    inbam.close()


#goes through a locations list 
def construct_df(locations, bam_file, del_lst, ins_lst, indel_lst):
    print('locations', locations)
    locations = locations.copy()    
    count = 0
    #iterates through the bams 
    for index, row in locations.iterrows():
        
        region = row['REGION']
        if count % 1000 == 0:
            print(count)
        count += 1

        inbam = pysam.AlignmentFile(bam_file, "rb")  # Store filename as key, BAM object as value
        
        chrom = locations[locations['REGION'] == region]['chrom'].tolist()[0]
        start = locations[locations['REGION'] == region]['start'].tolist()[0]
        end = locations[locations['REGION'] == region]['end'].tolist()[0]

        #modifying this so open BAMs etc
        deletions = del_lst[region]
        indels = indel_lst[region]
        
        deletion_lengths = [
            [deletion[1] for deletion in read if 'deletions' in deletion[0]]
            for read in deletions
        ]

        deletion_coords = [
            [deletion[2] for deletion in read if 'deletions' in deletion[0]]
            for read in deletions
        ]

        #keep track of the number of indels
        indel_lengths = [
            [indel[1] for indel in read if 'indel' in indel[0]]
            for read in indels
        ]
        
        num_deletions = len(deletion_lengths)

        deletion_extending_past_region_lengths = [deletion[1] for read in deletions for deletion in read if 'extending' in deletion[0]]
        deletion_extending_past_region_coords = [deletion[2] for read in deletions for deletion in read if 'extending' in deletion[0]]
        num_deletions_extending_past_region = len(deletion_extending_past_region_coords)
        
        insertions = ins_lst[region]
        
        #insertion_lengths = [insertion[1] for read in insertions for insertion in read if 'insertions' == insertion[0]]
        insertion_lengths = [
            [insertion[1] for insertion in read if 'insertions' in insertion[0]]
            for read in insertions 
        ]
        
        insertion_coords = [
            [insertion[2] for insertion in read if 'insertions' in insertion[0]]
            for read in insertions
        ]
        
        #insertion_coords = [insertion[2] for read in insertions for insertion in read if 'insertions' == insertion[0]]
        num_insertions = len(insertion_coords)
        num_indels = len(indel_lengths)

        #adding some columns to the dataframes
        total_reads = sum(1 for read in inbam.fetch(chrom, int(start), int(end)))
        
        locations.loc[locations['REGION'] == region, 'total_reads'] = total_reads
        locations.loc[locations['REGION'] == region, 'forward_reads'] = sum(1 for read in inbam.fetch(chrom, int(start), int(end)) if not read.is_reverse)
        locations.loc[locations['REGION'] == region, 'reverse_reads'] = sum(1 for read in inbam.fetch(chrom, int(start), int(end)) if read.is_reverse)

        if region == 'REGION_7126.bam':
            print(total_reads)
                    
        #collect info about all deletions
        locations.loc[locations['REGION'] == region, 'deletion lengths'] = str(deletion_lengths)
        locations.loc[locations['REGION'] == region, 'deletion coords'] = str(deletion_coords)
        locations.loc[locations['REGION'] == region, 'number of reads with deletion(s)'] = str(num_deletions)
        #collect info about all insertions
        locations.loc[locations['REGION'] == region, 'insertion lengths'] = str(insertion_lengths)
        locations.loc[locations['REGION'] == region, 'insertion coords'] = str(insertion_coords)
        locations.loc[locations['REGION'] == region, 'number of reads with insertion(s)'] = str(num_insertions)
        locations.loc[locations['REGION'] == region, 'number of reads with indels'] = str(num_indels)     


        #collect info about all deletions that go past the guide site
        locations.loc[locations['REGION'] == region, 'deletion extending past region lengths'] = str(deletion_extending_past_region_lengths)
        locations.loc[locations['REGION'] == region, 'deletion extending past region coords'] = str(deletion_extending_past_region_coords)
        locations.loc[locations['REGION'] == region, 'number of reads with deletion extending past region'] = str(num_deletions_extending_past_region)     

        if total_reads != 0:
            locations.loc[locations['REGION'] == region, '% reads with an deletion'] = int(num_deletions) / int(total_reads)
            locations.loc[locations['REGION'] == region, '% reads with an insertion'] = int(num_insertions) / int(total_reads)
            locations.loc[locations['REGION'] == region, '% reads with an indel'] = int(num_indels) / int(total_reads)
        else:
            locations.loc[locations['REGION'] == region, '% reads with an insertion'] = None
            locations.loc[locations['REGION'] == region, '% reads with an deletion'] = None
            locations.loc[locations['REGION'] == region, '% reads with an indel'] = None
        #close the bam file
        inbam.close()
    return locations

In [34]:
from Bio import pairwise2
from Bio.Seq import Seq
import logging
import warnings
import pysam

# Suppress warnings
logging.getLogger("pysam").setLevel(logging.ERROR)
warnings.filterwarnings("ignore", message="hts_idx_load3")
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.simplefilter("ignore", UserWarning)

guide_sequence = "TAGTAATAGTACCTAATGTG"
strandedness = []

#GOES THROUGH ALL THE TSV FILES (THAT CONTAIN INFORMATION ABOUT THE CHANGE SEQ LOCATIONS - AND EXTRACT INFORMATION ABOUT THE 
#STRANDNEDNESS (NOT REALLY SURE THIS INFORMATION IS RELEVANT - BUT DO IT ANYWAYS)

for condition in ['ALTC', 'ALT7', 'REXC', 'REX4']:
    print(condition)
    strandedness = []
    
    # for index, row in all_tsvs[condition].iterrows():
        
    #     # print(index)
    #     # if index == 8900:
    #     #     print(row)
        
    #     ref_sequence = fasta.fetch(row['chrom'], int(row['start']), int(row['end'])).upper()
    #     ref_complement = str(Seq(ref_sequence).reverse_complement())
        
    #     # Align guide sequence with reference and reference complement
    #     align_ref = pairwise2.align.globalxx(guide_sequence, ref_sequence)
    #     align_complement = pairwise2.align.globalxx(guide_sequence, ref_complement)
        
    #     # Calculate homology scores
    #     score_ref = align_ref[0][2] if align_ref else 0
    #     score_complement = align_complement[0][2] if align_complement else 0
        
    #     # Determine strandedness based on higher homology score
    #     if score_ref > score_complement:
    #         strand = '+'
    #     elif score_complement > score_ref:
    #         strand = '-'
    #     else:
    #         strand = "unknown"
        
    #     strandedness.append(strand)
    
    
    #not sure if this is accurate - the location of the off target on the strand is used to see where the deletion should be once aligned
    #the reads are reverse complemented (mention this)
    all_tsvs[condition]['strandedness'] = 'unknown'
    display(all_tsvs[condition])


ALTC


,chrom,start,end,name,na,na2,REGION,strandedness
0,chr3,81731576,81731598,NaN,NaN,NaN,REGION_0.bam,unknown
1,chr4,53055073,53055095,NaN,NaN,NaN,REGION_1.bam,unknown
2,chrX,54602603,54602626,NaN,NaN,NaN,REGION_2.bam,unknown
3,chr1,67440184,67440206,NaN,NaN,NaN,REGION_3.bam,unknown
4,chr7,152135028,152135050,NaN,NaN,NaN,REGION_4.bam,unknown
...,...,...,...,...,...,...,...,...
328,chr8,118489859,118489882,NaN,NaN,NaN,REGION_328.bam,unknown
329,chr21,30914949,30914972,NaN,NaN,NaN,REGION_329.bam,unknown
330,chr11,116585793,116585816,NaN,NaN,NaN,REGION_330.bam,unknown
331,chr11,11274660,11274683,NaN,NaN,NaN,REGION_331.bam,unknown


ALT7


,chrom,start,end,na,na2,REGION,strandedness
0,chr8,6125767,6125791,NaN,NaN,REGION_0.bam,unknown
1,chr1,153344375,153344398,NaN,NaN,REGION_1.bam,unknown
2,chr1,10813057,10813081,NaN,NaN,REGION_2.bam,unknown
3,chr7,100921222,100921246,NaN,NaN,REGION_3.bam,unknown
4,chr1,34737411,34737434,NaN,NaN,REGION_4.bam,unknown
...,...,...,...,...,...,...,...
2039,chr7,140179543,140179566,NaN,NaN,REGION_2039.bam,unknown
2040,chr6,20173116,20173139,NaN,NaN,REGION_2040.bam,unknown
2041,chr2,86022567,86022590,NaN,NaN,REGION_2041.bam,unknown
2042,chr2,75125534,75125556,NaN,NaN,REGION_2042.bam,unknown


REXC


,chrom,start,end,na,na2,REGION,strandedness
0,chr1,28594989,28595011,NaN,NaN,REGION_0.bam,unknown
1,chr15,94909979,94910002,NaN,NaN,REGION_1.bam,unknown
2,chr16,5625250,5625273,NaN,NaN,REGION_2.bam,unknown
3,chr5,135883747,135883770,NaN,NaN,REGION_3.bam,unknown
4,chr19,32985142,32985165,NaN,NaN,REGION_4.bam,unknown
...,...,...,...,...,...,...,...
2575,chr6,1610013,1610036,NaN,NaN,REGION_2575.bam,unknown
2576,chr1,36486968,36486991,NaN,NaN,REGION_2576.bam,unknown
2577,chrX,80805788,80805810,NaN,NaN,REGION_2577.bam,unknown
2578,chr1,1185685,1185708,NaN,NaN,REGION_2578.bam,unknown


REX4


,chrom,start,end,na,na2,REGION,strandedness
0,chr11,15300594,15300617,NaN,NaN,REGION_0.bam,unknown
1,chr21,45329598,45329621,NaN,NaN,REGION_1.bam,unknown
2,chr5,167196910,167196933,NaN,NaN,REGION_2.bam,unknown
3,chr20,51269529,51269552,NaN,NaN,REGION_3.bam,unknown
4,chr1,56258174,56258196,NaN,NaN,REGION_4.bam,unknown
...,...,...,...,...,...,...,...
177,chrX,56588923,56588946,NaN,NaN,REGION_177.bam,unknown
178,chr12,123801871,123801895,NaN,NaN,REGION_178.bam,unknown
179,chr13,24241712,24241735,NaN,NaN,REGION_179.bam,unknown
180,chr12,68285106,68285129,NaN,NaN,REGION_180.bam,unknown


In [ ]:
all_del_dict = {'ALTC':[{},{}],'ALT7':[{},{}],'REXC':[{},{}],'REX4':[{},{}]}
all_ins_dict = {'ALTC':[{},{}],'ALT7':[{},{}],'REXC':[{},{}],'REX4':[{},{}]}
all_ind_dict = {'ALTC':[{},{}],'ALT7':[{},{}],'REXC':[{},{}],'REX4':[{},{}]}

#goes through all conditions, all bams and all replicates - extracts each region analyzed and populates dictionary with
#seems like all the issues are with the AltC2 replicate

for condition in ['REXC', 'REX4', 'ALTC', 'ALT7']:
    condition_bams = all_bams[condition]
    tsv = all_tsvs[condition]    
    #display(tsv)
    replicate_num = 0

    #go through each replicate
    for replicate in condition_bams:
        count = 0
        inbam = replicate      
        for region in all_tsvs[condition]['REGION']:         
            bam_basename = ''
    
            if count % 1000 == 0:
                print(count)

            count += 1

            #how to properly extract the coordinates?
            chrom = tsv[tsv['REGION'] == region]['chrom'].tolist()[0]
            start = tsv[tsv['REGION'] == region]['start'].tolist()[0]
            end = tsv[tsv['REGION'] == region]['end'].tolist()[0]

            strand = 'unknown'
            bam_file = pysam.AlignmentFile(inbam, "rb") 

            #instead of passing in the bam_file itself passing in a name to the bam_file and then should close it
            all_del_dict[condition][replicate_num][region],all_ins_dict[condition][replicate_num][region], all_ind_dict[condition][replicate_num][region] = find_deletion_locs(bam_file, chrom, int(start), int(end), strand)
                #close the bam_file    
        
            bam_file.close()  # Close the BAM file
        replicate_num += 1

0


In [ ]:
## all_del_dict_wt = {'ALTC':{},'ALT7':{},'REXC':{},'REX4':{}}
all_ins_dict_wt = {'ALTC':{},'ALT7':{},'REXC':{},'REX4':{}}
all_ind_dict_wt = {'ALTC':{},'ALT7':{},'REXC':{},'REX4':{}}

#goes through all conditions, all bams and all replicates - extracts each region analyzed and gets the insertion etc
#this is just for the control subset

#seems to be appropriatly populating
for condition in ["ALTC", "REXC", "REX4", "ALT7"]:
    condition_bams = all_controls[condition]

    count = 0

    display(all_tsvs[condition])
    
    inbam = condition_bams    
    for region in all_tsvs[condition]['REGION']:
        if count % 1000 == 0:
            print(condition)
            print(count)
                
        count += 1

        chrom = all_tsvs[condition][all_tsvs[condition]['REGION'] == region]['chrom'].tolist()[0]
        start = all_tsvs[condition][all_tsvs[condition]['REGION'] == region]['start'].tolist()[0]
        end = all_tsvs[condition][all_tsvs[condition]['REGION'] == region]['end'].tolist()[0]
                    
        #open the bam_file
        bam_file = pysam.AlignmentFile(inbam, "rb") 
        #instead of passing in the bam_file itself passing in a name to the bam_file and then should close it
        
        all_del_dict_wt[condition][region],all_ins_dict_wt[condition][region], all_ind_dict_wt[condition][region] = find_deletion_locs(bam_file, chrom, int(start), int(end), strand)
        bam_file.close()  # Close the BAM file

In [ ]:
all_del_dict_wt['ALTC']

In [ ]:
 edited_dfs[condition][replicate]

In [ ]:
edited_dfs = {'ALTC': [None, None],
      'ALT7': [None, None],
      'REXC': [None, None],
      'REX4':[None, None]}

control_dfs = {'ALTC': None,
      'ALT7': None,
      'REXC': None,
      'REX4': None}

#iterate through the conditions (ALTC, ALT7, REX4, and REXC) and then create tables
for condition in ['ALTC', 'ALT7', 'REXC', 'REX4']:
    print(all_bams[condition][replicate])
    for replicate in np.arange(0,2):
        edited_dfs[condition][replicate] = construct_df(all_tsvs[condition],all_bams[condition][replicate], all_del_dict[condition][replicate],
                     all_ins_dict[condition][replicate], all_ind_dict[condition][replicate])

In [ ]:
#construct all the dataframes for the insertions/deletions/indels present in the control (no nucleoinfection) group
#in this case there are NOT replicates
for condition in ['ALTC', 'ALT7', 'REXC', 'REX4']:
    control_dfs[condition] = construct_df(all_tsvs[condition],all_controls[condition], all_del_dict_wt[condition],
                 all_ins_dict_wt[condition], all_ind_dict_wt[condition])

In [ ]:
for condition in ['ALTC', 'ALT7', 'REXC', 'REX4']:
    for replicate in  np.arange(0,2):
        df = edited_dfs[condition][replicate]
        display(df)
        print(condition)
        print(replicate)
        df.to_csv(str(condition) + 'new_concatanted_taylor_consensus_indel_rates_hg38' + str(replicate) + '.csv', index=False)  # Set index=True if you want to keep the index


for condition in ['ALTC','ALT7', 'REXC', 'REX4']:
    df = control_dfs[condition]
    print(condition)
    display(df)
    df.to_csv(str(condition) + 'new_concatanted_taylor_consensus_indel_rates_control_hg38' + '.csv', index=False)  # Set index=True if you want to keep the index

In [ ]:
#read in the csvs and populate tables so don't have to redo it all 

# # edited_dfs = {'ALTC': [None, None],
# #       'ALT7': [None, None],
# #       'REXC': [None, None],
# #       'REX4':[None, None]}

# # control_dfs = {'ALTC': None,
# #       'ALT7': None,
# #       'REXC': None,
#       'REX4': None}

# for condition in ['ALTC', 'ALT7', 'REXC', 'REX4']:
#     for replicate in  np.arange(0,2):
#         file_path = f"{condition}consensus_indel_rates_hg38_new_con{replicate}.csv"  # Construct file name
#         print(file_path)
#         edited_dfs[condition][replicate] = pd.read_csv(file_path)

# for condition in ['ALTC','ALT7', 'REXC', 'REX4']:
#     file_path = f"{condition}consensus_indel_rates_control_hg38_new_con.csv"  # Using f-string for readability
#     print(file_path)
#     control_dfs['ALTC'] = pd.read_csv(file_path)


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import fisher_exact
import scipy.stats as stats


def construct_comp_tbl(wt, edited):
    comp_df = pd.DataFrame([])

    # Ensure 'REGION' column is included
    comp_df['REGION'] = wt['REGION'].tolist()
    

    #construct a contigency table
    for index, row in wt.iterrows():
        # display(row)
        cont = pd.DataFrame([])
        cont_indel = pd.DataFrame([])
        
        region = row['REGION']
        cont['wt'] = [int(row['number of reads with indels']),
        int(row['total_reads']) - int(row['number of reads with indels'])]

        row2 = edited.iloc[index]
        #display(row2)
        cont['edited'] = [int(row2['number of reads with indels']),
        int(row2['total_reads']) - int(row2['number of reads with indels'])]
        
        cont['type'] = ['indel', 'no indel']
        cont.set_index('type', inplace=True)
        
        # display(cont)
        #not quite significant -- perhaps because don't have enough depth
        #now that we have constructed the contigency tables - do some stats !
        odds_ratio, p_value = stats.fisher_exact(cont)
        #print(odds_ratio,p_value)

        
        # #create the regions table
        # #comp_df.loc[comp_df['REGION'] == region, 'deletion odds_ratio'] = odds_ratio
        # comp_df.loc[comp_df['REGION'] == region, 'deletion p-value'] = p_value
        
        # comp_df.loc[comp_df['REGION'] == region, 'indel odds_ratio'] = odds_ratio

        comp_df.loc[comp_df['REGION'] == region, 'chrom'] = row['chrom']
        comp_df.loc[comp_df['REGION'] == region, 'start'] = row2['start']
        comp_df.loc[comp_df['REGION'] == region, 'end'] = row['end']
        
        comp_df.loc[comp_df['REGION'] == region, 'edited indel %'] = row2['% reads with an indel']
        comp_df.loc[comp_df['REGION'] == region, 'wt indel %'] = row['% reads with an indel']
        comp_df.loc[comp_df['REGION'] == region, 'edited indel %'] = row2['% reads with an indel']
        comp_df.loc[comp_df['REGION'] == region, 'total edited reads'] = row2['total_reads']
        comp_df.loc[comp_df['REGION'] == region, 'total wt reads'] = row['total_reads']
        comp_df.loc[comp_df['REGION'] == region, 'edited reads with indels '] = row2['number of reads with indels']
        comp_df.loc[comp_df['REGION'] == region, 'wt reads with indels'] = row['number of reads with indels']
        comp_df.loc[comp_df['REGION'] == region, 'indel p-value'] = p_value
        
    return comp_df

In [ ]:
#can do it for each replicate  - and then join on region
comparison_tables = {'ALTC': [None, None],
                     'ALT7': [None, None],
                     'REX4': [None, None],
                     'REXC': [None, None]}

for condition in ['ALT7', 'REXC', 'REX4', 'ALTC']:
    for replicate in [0,1]:
        print('CONDITION', condition)
        # display(control_dfs[condition])
        # display(edited_dfs[condition][replicate])
        display(control_dfs[condition])
        comparison_tables[condition][replicate] = construct_comp_tbl(control_dfs[condition], edited_dfs[condition][replicate])
        comparison_tables[condition][replicate]['trial'] = replicate
        print('done with a condition')

In [ ]:
#can do it for each replicate  - and then join on region
comparison_tables = {'ALTC': [None, None],
                     'ALT7': [None, None],
                     'REX4': [None, None],
                     'REXC': [None, None]}

for condition in ['ALTC', 'ALT7', 'REXC', 'REX4']:
    for replicate in [0,1]:
        print('CONDITION', condition)
        # display(control_dfs[condition])
        # display(edited_dfs[condition][replicate])
        display(control_dfs[condition])
        comparison_tables[condition][replicate] = construct_comp_tbl(control_dfs[condition], edited_dfs[condition][replicate])
        comparison_tables[condition][replicate]['trial'] = replicate
        print('done with a condition')

In [ ]:
# Fill all NaNs with 0 in both DataFrames
edited_dfs['ALTC'][0] = edited_dfs['ALTC'][0].fillna(0)
edited_dfs['ALTC'][1] = edited_dfs['ALTC'][1].fillna(0)

control_dfs['ALTC'] = control_dfs['ALTC'].fillna(0)

edited_dfs['ALTC'][1][edited_dfs['ALTC'][1]['REGION'] == 'REGION_15.bam']
edited_dfs['ALTC'][0][edited_dfs['ALTC'][0]['REGION'] == 'REGION_15.bam']


In [ ]:
for condition in ['ALTC', 'ALT7', 'REXC', 'REX4']:
    for replicate in np.arange(0,2):
        # display(edited_dfs[condition][replicate])
        #display(comparison_tables[condition][replicate])
        print(f"{condition}_{replicate}casoffinder_taylor_consensus_comparison_hg38.csv")
        comparison_tables[condition][replicate].to_csv(f"{condition}{replicate}casoffinder_taylor_consensus_comparison_hg38.csv")

In [ ]:
comparison_tables['ALTC'][1]

In [ ]:
#read in the CSVs from comparison into the dictionary (access again!)

# for condition in ['ALT7', 'ALTC', 'REXC', 'REX4']:
#     for replicate in np.arange(0,2):
#         print('CONDITION', condition)
#         # display(control_dfs[condition])
#         # display(edited_dfs[condition][replicate])
#         comparison_tables[condition][replicate] = pd.read_csv(f"{condition}_{replicate}consensus_comparison_hg38_con_filtered.csv")
#         comparison_tables[condition][replicate]['trial'] = replicate

In [ ]:
# display(comparison_tables['ALTC'][1])

In [ ]:
#can do a combination - merge the dataframes together for the replicates
#add a new row that gives the condition - so that when you join the two tables there is something that distinguishes the 2 replicates

merged_comparison_tables = {'ALTC': None,
                     'ALT7': None,
                     'REX4': None,
                     'REXC': None}

for condition in ['ALTC', 'ALT7', 'REXC', 'REX4']:
    print('CONDITION', condition)
    merged_comparison_tables[condition] = pd.concat([comparison_tables[condition][0], comparison_tables[condition][1]])
    display(merged_comparison_tables[condition][merged_comparison_tables[condition]['REGION'] == 'REGION_0.bam'])
    print((merged_comparison_tables[condition].shape[0]))
    display(comparison_tables[condition][0])
    display(comparison_tables[condition][1])
    merged_comparison_tables[condition].to_csv(f"{condition}taylor_consensus_merged_comparisond_con_new_hg38_filtered_mp20.csv")

In [ ]:
# merged_comparison_tables['ALTC']

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Create the figure


## chr9:27575447-27575470 (ALT7) REGION_88.bam


# chr9:27573509-27573532 (REXC) REGION_78.bam	
# chr9:27575447-27575470 (ALT7) REGION_88.bam
# chr9:27573647-27573670 (REX4) REGION_14.bam

def plot_scatter(tbl):
    plt.figure(figsize=(8,6))

    # Color the first point differently by separating it
    sns.scatterplot(
        x=tbl["wt indel %"].to_list()[1:],  # Exclude first point
        y=tbl["edited indel %"].to_list()[1:],  # Exclude first point
        color='black',  # Makes the rest of the points black
        alpha=0.5
    )
    
    # Plot the first point with a different color (e.g., red)
    sns.scatterplot(
        x=[tbl["wt indel %"].to_list()[167]],  # First point
        y=[tbl["edited indel %"].to_list()[167]],  # First point
        color='red',  # Different color for the first point
        s=100,  # Increase size of first point
        label="Edited Region (ALT7 Guide - chr9:27575447-27575470 )"  # Optional: add label for legend
    )
    
    # Labels and title
    plt.xlabel("Non-Edited Indel Percentage (%)", fontsize=12)  # X-axis label
    plt.ylabel("Edited Indel Percentage (%)", fontsize=12)  # Y-axis label
    plt.title("Indel Percentage Comparison: Non-Edited vs. Edited (Replicate 1)", fontsize=14)  # Title
    
    # Show the plot
    plt.legend()  # Optional: to show the label in the legend
    plt.show()


#tbl = merged_comparison_tables['ALT7'][merged_comparison_tables['ALT7']['trial'] == 0]

tbl = comparison_tables['REXC'][0]
display(tbl)
plot_scatter(tbl)

#display(comparison_tables['REX4'][0][comparison_tables['REX4'][0]['REGION' == 'REGION_15.bam']])
display(comparison_tables['ALT7'][1].head(15))


#comparison_tables['ALTC'][0][comparison_tables['ALTC'][0]['chrom'] == 'chr9'].head(30)

In [ ]:
merged_comparison_tables['REX4'][merged_comparison_tables['REX4']['REGION'] == 'REGION_14.bam']
display(edited_dfs['REX4'][0][edited_dfs['REX4'][0]['REGION'] == 'REGION_14.bam'])
display(control_dfs['REX4'][control_dfs['REX4']['REGION'] == 'REGION_14.bam'])

In [ ]:
display(edited_dfs['REXC'][0][edited_dfs['REXC'][0]['REGION'] == 'REGION_78.bam'])
display(control_dfs['REXC'][control_dfs['REXC']['REGION'] == 'REGION_78.bam'])

In [ ]:
print([i for i in edited_dfs['REXC'][0][edited_dfs['REXC'][0]['REGION'] == 'REGION_78.bam']['deletion lengths'].to_list()])
print(control_dfs['REXC'][control_dfs['REXC']['REGION'] == 'REGION_78.bam']['deletion lengths'].to_list())

#why are there so many 6 bp deletions??


In [ ]:
import ast  # To safely evaluate string representations of lists

# Extract deletion lengths
edited_raw = edited_dfs['REXC'][0][edited_dfs['REXC'][0]['REGION'] == 'REGION_78.bam']['deletion lengths'].to_list()
control_raw = control_dfs['REXC'][control_dfs['REXC']['REGION'] == 'REGION_78.bam']['deletion lengths'].to_list()

# Convert possible string representations of lists to actual lists
if isinstance(edited_raw, str):  
    edited_raw = ast.literal_eval(edited_raw)  
if isinstance(control_raw, str):  
    control_raw = ast.literal_eval(control_raw)  

# Flatten the lists
edited_deletion_lengths = [item[0] if isinstance(item, list) else item for item in edited_raw]
control_deletion_lengths = [item[0] if isinstance(item, list) else item for item in control_raw]

print("Edited Deletion Lengths:", np.array(edited_deletion_lengths[0]))
print("Control Deletion Lengths:", control_deletion_lengths[0])

In [ ]:
import ast  # Used to safely evaluate string representations of lists

# Extract the raw data from the DataFrame
edited_raw = edited_dfs['REXC'][0][edited_dfs['REXC'][0]['REGION'] == 'REGION_78.bam']['deletion lengths'].to_list()

# If the first item is a string, convert it to a list
if isinstance(edited_raw, list) and len(edited_raw) == 1 and isinstance(edited_raw[0], str):
    edited_raw = ast.literal_eval(edited_raw[0])  # Convert string to a proper list

# Now flatten the nested list
edited_deletion_lengths = [x[0] for x in edited_raw]  # Extract numbers from single-item lists

print(edited_deletion_lengths)  # Now it's a proper list of numbers

import ast  # Used to safely evaluate string representations of lists

# Extract the raw data from the DataFrame
control_raw = control_dfs['REXC'][control_dfs['REXC']['REGION'] == 'REGION_78.bam']['deletion lengths'].to_list()

# If the first item is a string, convert it to a list
if isinstance(control_raw, list) and len(control_raw) == 1 and isinstance(control_raw[0], str):
    control_raw = ast.literal_eval(control_raw[0])  # Convert string to a proper list

# Now flatten the nested list
control_deletion_lengths = [x[0] for x in control_raw]  # Extract numbers from single-item lists

print(control_deletion_lengths)  # Now it's a proper list of numbers



In [ ]:
import matplotlib.pyplot as plt
import ast
import numpy as np  # For histogram calculations

# Extract the raw data from the DataFrame for CONTROL
control_raw = control_dfs['REX4'][control_dfs['REX4']['REGION'] == 'REGION_14.bam']['deletion lengths'].to_list()
if isinstance(control_raw, list) and len(control_raw) == 1 and isinstance(control_raw[0], str):
    control_raw = ast.literal_eval(control_raw[0])  # Convert string to a proper list
control_deletion_lengths = [x[0] for x in control_raw]  # Extract numbers from single-item lists

# Extract the raw data from the DataFrame for EDITED (1)
edited_raw_1 = edited_dfs['REX4'][0][edited_dfs['REX4'][0]['REGION'] == 'REGION_14.bam']['deletion lengths'].to_list()
if isinstance(edited_raw_1, list) and len(edited_raw_1) == 1 and isinstance(edited_raw_1[0], str):
    edited_raw_1 = ast.literal_eval(edited_raw_1[0])  # Convert string to a proper list
edited_deletion_lengths_1 = [x[0] for x in edited_raw_1]  # Extract numbers from single-item lists

# Extract the raw data from the DataFrame for EDITED (2)
edited_raw_2 = edited_dfs['REX4'][1][edited_dfs['REX4'][1]['REGION'] == 'REGION_14.bam']['deletion lengths'].to_list()
if isinstance(edited_raw_2, list) and len(edited_raw_2) == 1 and isinstance(edited_raw_2[0], str):
    edited_raw_2 = ast.literal_eval(edited_raw_2[0])  # Convert string to a proper list
edited_deletion_lengths_2 = [x[0] for x in edited_raw_2]  # Extract numbers from single-item lists

# Calculate the bin range for all data
all_deletion_lengths = edited_deletion_lengths_1 + edited_deletion_lengths_2 + control_deletion_lengths
bin_range = range(min(all_deletion_lengths), max(all_deletion_lengths) + 1)
bin_width = 1  # Set the width of the bins

# Compute histogram values
control_hist, bin_edges = np.histogram(control_deletion_lengths, bins=bin_range)
edited_hist_1, _ = np.histogram(edited_deletion_lengths_1, bins=bin_range)
edited_hist_2, _ = np.histogram(edited_deletion_lengths_2, bins=bin_range)

# Set the positions for each dataset's bars
width = 0.3  # Bar width
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2  # Get the center of each bin for placing bars

# Create the combined histogram with bars side by side
plt.figure(figsize=(10, 5))

# Plot the bars for the control dataset
plt.bar(bin_centers - width, control_hist, width=width, alpha=0.7, color='red', edgecolor='black', label='Control')

# Plot the bars for the first edited dataset
plt.bar(bin_centers, edited_hist_1, width=width, alpha=0.7, color='blue', edgecolor='black', label='Edited 1')

# Plot the bars for the second edited dataset
plt.bar(bin_centers + width, edited_hist_2, width=width, alpha=0.7, color='cyan', edgecolor='black', label='Edited 2')

# Labels and title
plt.xlabel("Deletion Length (bp)")
plt.ylabel("Frequency")
plt.title("Deletion Lengths On Target Region - REX4")
plt.legend()  # Add legend to differentiate between datasets

# Show the plot
plt.show()

